In [1]:
import glob

In [2]:
cat data/bvlcalexnet-7.onnx/145.40.69.165/perf_output.txt


Start computing 1 iterations
Finish computing 1 iterations

 Performance counter stats for './bvlcalexnet-7.onnx.bin':

            524.67 msec task-clock                #    1.000 CPUs utilized          
        2572120980      cycles                    #    4.902 GHz                    
        5380881682      instructions              #    2.09  insn per cycle         
          79796834      cache-references          #  152.089 M/sec                  
           5647912      cache-misses              #    7.078 % of all cache refs    
         709814933      branches                  # 1352.875 M/sec                  
           1039967      branch-misses             #    0.15% of all branches        

       0.524810221 seconds time elapsed

       0.512840000 seconds user
       0.012019000 seconds sys




In [3]:
success_phrase = """Start computing 1 iterations
Finish computing 1 iterations"""

In [4]:
failures = []
successes = []
models = []
hosts = []

In [5]:
from pathlib import Path

In [6]:
p = Path("data/bvlcalexnet-7.onnx/145.40.69.165/perf_output.txt")


In [7]:
p.parts[-3]

'bvlcalexnet-7.onnx'

In [8]:
for fn in glob.iglob("data/*/*/perf_output.txt"):
    fn = Path(fn)
    
    with open(fn, mode="r") as f:
        contents=f.read()
        if success_phrase not in contents:
            failures.append(contents)
        else:
            successes.append(contents)
            models.append(fn.parts[-3])
            hosts.append(fn.parts[-2])

In [9]:
def parse(contents):
    d = {}
    for line in contents.split("\n")[5:]:
        parts = line.split("#")
        for part in parts:
            words = part.split()
            if len(words)>0:
                k = " ".join(words[1:])

                number_string = words[0]
                if number_string[-1] == "%":
                    v = float(number_string[:-1])/100
                else:
                    v = float(number_string)
                
                d[k]=v
    return d

In [10]:
records = []

In [11]:
for contents, host, model in zip(successes, hosts, models):
    d = parse(contents)
    d["host"] = host
    d["model"]= model
    records.append(d)

In [12]:
import pandas as pd

In [13]:
df_raw = pd.DataFrame.from_records(records)

In [14]:
df_raw

,msec task-clock,CPUs utilized,cycles,GHz,instructions,insn per cycle,cache-references,M/sec,cache-misses,% of all cache refs,branches,branch-misses,of all branches,seconds time elapsed,seconds user,seconds sys,host,model
0,954.08,1.000,3.164130e+09,3.316,5.934072e+09,1.88,5.247691e+08,824.483,26231599.0,4.999,7.866217e+08,7729330.0,0.0098,0.954343,0.946285,0.008019,93.187.218.93,rcnn-ilsvrc13-7.onnx
1,939.60,0.999,2.739255e+09,2.915,5.934294e+09,2.17,5.257923e+06,837.232,4143852.0,78.812,7.866668e+08,1127138.0,0.0014,0.940093,0.932194,0.008001,147.75.48.193,rcnn-ilsvrc13-7.onnx
2,946.99,0.999,3.133359e+09,3.309,5.934516e+09,1.89,5.235067e+08,830.757,23788451.0,4.544,7.867177e+08,7747491.0,0.0098,0.947744,0.943293,0.004014,147.28.128.247,rcnn-ilsvrc13-7.onnx
3,585.70,1.000,2.782024e+09,4.750,5.966425e+09,2.14,7.297117e+07,1349.913,5317091.0,7.287,7.906461e+08,1116730.0,0.0014,0.585905,0.581907,0.004013,145.40.69.165,rcnn-ilsvrc13-7.onnx
4,949.79,1.000,3.159559e+09,3.327,5.934091e+09,1.88,5.489181e+08,828.206,26327077.0,4.796,7.866257e+08,7686908.0,0.0098,0.950063,0.942000,0.008017,93.187.218.93,rcnn-ilsvrc13-6.onnx
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,15602.29,1.000,7.758953e+10,4.973,2.039151e+11,2.63,1.084045e+10,1768.891,17182511.0,0.159,2.759874e+10,12572911.0,0.0005,15.602797,15.574609,0.027997,145.40.69.165,vgg19-bn-7.onnx
356,6805.42,1.000,2.257900e+10,3.318,2.629949e+10,1.16,2.658240e+09,134.675,9278653.0,0.349,9.165208e+08,5318887.0,0.0058,6.806756,6.785638,0.020004,93.187.218.93,rain-princess-8.onnx
357,9851.65,1.000,3.029029e+10,3.075,2.630633e+10,0.87,6.783535e+07,93.156,3007188.0,4.433,9.177398e+08,8226932.0,0.0090,9.852266,9.836315,0.016000,147.75.48.193,rain-princess-8.onnx
358,6774.22,1.000,2.260324e+10,3.337,2.630216e+10,1.16,2.657416e+09,135.384,15106729.0,0.568,9.171201e+08,5330166.0,0.0058,6.776973,6.753681,0.019993,147.28.128.247,rain-princess-8.onnx


In [15]:
set(df_raw.model)

{'ResNet101-DUC-7.onnx',
 'arcfaceresnet100-8.onnx',
 'bvlcalexnet-6.onnx',
 'bvlcalexnet-7.onnx',
 'bvlcalexnet-8.onnx',
 'bvlcalexnet-9.onnx',
 'caffenet-6.onnx',
 'caffenet-7.onnx',
 'caffenet-8.onnx',
 'caffenet-9.onnx',
 'candy-8.onnx',
 'candy-9.onnx',
 'densenet-6.onnx',
 'densenet-7.onnx',
 'densenet-8.onnx',
 'densenet-9.onnx',
 'efficientnet-lite4-11.onnx',
 'emotion-ferplus-7.onnx',
 'emotion-ferplus-8.onnx',
 'googlenet-3.onnx',
 'googlenet-6.onnx',
 'googlenet-7.onnx',
 'googlenet-8.onnx',
 'googlenet-9.onnx',
 'inception-v1-6.onnx',
 'inception-v1-7.onnx',
 'inception-v1-8.onnx',
 'inception-v1-9.onnx',
 'inception-v2-7.onnx',
 'inception-v2-8.onnx',
 'inception-v2-9.onnx',
 'mnist-7.onnx',
 'mnist-8.onnx',
 'mosaic-8.onnx',
 'mosaic-9.onnx',
 'pointilism-8.onnx',
 'pointilism-9.onnx',
 'rain-princess-8.onnx',
 'rain-princess-9.onnx',
 'rcnn-ilsvrc13-6.onnx',
 'rcnn-ilsvrc13-7.onnx',
 'rcnn-ilsvrc13-8.onnx',
 'rcnn-ilsvrc13-9.onnx',
 'resnet101-v1-7.onnx',
 'resnet101-v2-

In [16]:
accepted_models=[ 
 'resnet101-v2-7.onnx',
 'resnet152-v2-7.onnx',
 'resnet18-v2-7.onnx',
 'resnet34-v2-7.onnx',
 'resnet50-v2-7.onnx',
]

In [17]:
set(df_raw.host)

{'145.40.69.165', '147.28.128.247', '147.75.48.193', '93.187.218.93'}

In [41]:
def host_map(host):
    m = {'145.40.69.165':0 , '147.28.128.247': 2, '147.75.48.193':3 , '93.187.218.93':1}
    return m[host]


In [42]:
def model_map(model):
    if "resnet" in model:
        return int(model.split("-")[0][6:])

In [44]:
df = df_raw[df_raw.model.isin(accepted_models)].copy()

In [45]:
df["x"] = df.host.map(host_map)
df["y"] = df.model.map(model_map)

In [46]:
df["z"] = df["msec task-clock"]

In [47]:
import numpy as np

In [48]:
df = (df-df.mean())/df.std()

/tmp/ipykernel_3213087/2345797706.py:1: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



In [49]:
from sklearn.neural_network import MLPRegressor

In [50]:
import plotly.express as px

In [ ]:
import chart_studio.plotly as py
import chart_studio.plotly.graph_objects as go


ModuleNotFoundError: No module named 'chart_studio.plotly.graph_objects'

In [ ]:
chart_studio.plotly.

In [53]:
import chart_studio

chart_studio.tools.set_credentials_file(username='cedar_ren', api_key='X8tRodKST4CoNSnsi9mY')

In [56]:

mesh_size = 0.1
margin = 0

X = df[['x', 'y']]
y = df['z']

# Condition the model on sepal width and length, predict the petal width
model = MLPRegressor(hidden_layer_sizes=[30,30,10],)
model.fit(X, y)

# Create a mesh grid on which we will run our model
x_min, x_max = X.x.min() - margin, X.x.max() + margin
y_min, y_max = X.y.min() - margin, X.y.max() + margin
xrange = np.arange(x_min, x_max, mesh_size)
yrange = np.arange(y_min, y_max, mesh_size)
xx, yy = np.meshgrid(xrange, yrange)

# Run model
pred = model.predict(np.c_[xx.ravel(), yy.ravel()])
pred = pred.reshape(xx.shape)

# Generate the plot
fig = px.scatter_3d(df, x='x', y='y', z='z')
fig.update_traces(marker=dict(size=5))
fig.add_traces(go.Surface(x=xrange, y=yrange, z=pred, name='pred_surface'))
fig.show()


In [57]:
import chart_studio.plotly as py
py.plot(fig, filename = 'swe_map_test', auto_open=True)

'https://plotly.com/~cedar_ren/35/'